In [9]:
import mysql.connector
import pandas as pd
import logging
from datetime import datetime

# -------------------------------
# تنظیمات
# -------------------------------
EXCEL_FILE = "abnormal_results_mhi_lube_oil_g11.xlsx"
LOG_FILE = "export_abnormal.log"

# تنظیم لاگ
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE, encoding='utf-8'),
        logging.StreamHandler()
    ]
)

# -------------------------------
# 1. اتصال به MySQL
# -------------------------------
def get_connection():
    return mysql.connector.connect(
        host='127.0.0.1',
        port=3306,
        user='root',
        password='',
        database='dsas'
    )

# -------------------------------
# 2. تابع اصلی: استخراج و ذخیره Abnormal
# -------------------------------
def export_abnormal_to_excel():
    conn = None
    try:
        conn = get_connection()
        query = """
            SELECT 
                `id`, `AssetID_8341`, `AssetID_8342`, `AssetID_8343`, `AssetID_8344`,
                `AssetID_8346`, `AssetID_9286`, `AssetID_9287`, `unitID`,
                `model_name`, `model_result`, `model_value`,
                `DateTime`, `RecordDate`, `RecordTime`, `TimeStamps`,
                `created_at`, `updated_at`
            FROM `results_table_mhi_lube_oil_g11`
            WHERE `model_result` = 'Abnormal'
            ORDER BY `TimeStamps` DESC
        """
        
        logging.info("در حال خواندن رکوردهای Abnormal از جدول...")
        df = pd.read_sql(query, conn)

        if df.empty:
            logging.info("هیچ رکوردی با model_result = 'Abnormal' یافت نشد.")
            print("هیچ مورد غیرعادی یافت نشد.")
            return

        # ذخیره در اکسل
        df.to_excel(EXCEL_FILE, index=False)
        logging.info(f"{len(df)} رکورد Abnormal با موفقیت در '{EXCEL_FILE}' ذخیره شد.")
        print(f"موفقیت‌آمیز: {len(df)} مورد غیرعادی در '{EXCEL_FILE}' ذخیره شد.")

    except mysql.connector.Error as e:
        logging.error(f"خطا در اتصال به دیتابیس یا اجرای کوئری: {e}")
        print(f"خطا در دیتابیس: {e}")
    except Exception as e:
        logging.error(f"خطای غیرمنتظره: {e}")
        print(f"خطای غیرمنتظره: {e}")
    finally:
        if conn and conn.is_connected():
            conn.close()
            logging.info("اتصال به دیتابیس بسته شد.")

# -------------------------------
# 3. اجرا
# -------------------------------
if __name__ == "__main__":
    start_time = datetime.now()
    logging.info("شروع استخراج رکوردهای Abnormal...")
    print("در حال استخراج رکوردهای غیرعادی...")
    
    export_abnormal_to_excel()
    
    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
    logging.info(f"پایان فرآیند. مدت زمان: {duration:.2f} ثانیه")
    print(f"تمام شد. مدت زمان: {duration:.2f} ثانیه")

2025-11-14 15:52:47,320 | INFO | شروع استخراج رکوردهای Abnormal...
2025-11-14 15:52:47,324 | INFO | در حال خواندن رکوردهای Abnormal از جدول...
C:\Users\pishva_r\AppData\Local\Temp\ipykernel_16308\2598748301.py:54: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
2025-11-14 15:52:47,397 | INFO | 1 رکورد Abnormal با موفقیت در 'abnormal_results_mhi_lube_oil_g11.xlsx' ذخیره شد.
2025-11-14 15:52:47,399 | INFO | اتصال به دیتابیس بسته شد.
2025-11-14 15:52:47,401 | INFO | پایان فرآیند. مدت زمان: 0.08 ثانیه


در حال استخراج رکوردهای غیرعادی...
موفقیت‌آمیز: 1 مورد غیرعادی در 'abnormal_results_mhi_lube_oil_g11.xlsx' ذخیره شد.
تمام شد. مدت زمان: 0.08 ثانیه


In [ ]:
import mysql.connector
import pandas as pd
import numpy as np
import logging
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM

# -------------------------------
# تنظیمات
# -------------------------------
EXCEL_FILE = "abnormal_results_mhi_lube_oil_g11_double_filtered.xlsx"
LOG_FILE = "export_abnormal_double_filter.log"

# تنظیم لاگ
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    handlers=[
        logging.FileHandler(LOG_FILE, encoding='utf-8'),
        logging.StreamHandler()
    ]
)

# -------------------------------
# 1. اتصال به MySQL
# -------------------------------
def get_connection():
    return mysql.connector.connect(
        host='127.0.0.1',
        port=3306,
        user='root',
        password='',
        database='dsas'
    )

# -------------------------------
# 2. تابع اصلی: دو مرحله فیلتر + One-Class SVM
# -------------------------------
def export_double_filtered_abnormal():
    conn = None
    try:
        conn = get_connection()

        # مرحله ۱: فقط رکوردهای Abnormal اولیه
        query = """
            SELECT 
                `id`, `AssetID_8341`, `AssetID_8342`, `AssetID_8343`, `AssetID_8344`,
                `AssetID_8346`, `AssetID_9286`, `AssetID_9287`, `unitID`,
                `model_name`, `model_result`, `model_value`,
                `DateTime`, `RecordDate`, `RecordTime`, `TimeStamps`,
                `created_at`, `updated_at`
            FROM `results_table_mhi_lube_oil_g11`
            WHERE `model_result` = 'Abnormal'
            ORDER BY `TimeStamps` DESC
        """
        
        logging.info("در حال خواندن رکوردهای Abnormal اولیه از جدول...")
        df = pd.read_sql(query, conn)

        if df.empty:
            logging.info("هیچ رکوردی با model_result = 'Abnormal' یافت نشد.")
            print("هیچ مورد غیرعادی اولیه یافت نشد.")
            return

        logging.info(f"{len(df)} رکورد Abnormal اولیه استخراج شد.")

        # ------------------------------------------------------------------
        # مرحله ۲: اعمال One-Class SVM روی model_value و سایر ویژگی‌های عددی
        # ------------------------------------------------------------------
        logging.info("آماده‌سازی داده‌ها برای One-Class SVM...")

        # انتخاب ویژگی‌های عددی مهم (می‌تونی بر حسب دیتای تو تغییر بده)
        feature_cols = [
            'AssetID_8341', 'AssetID_8342', 'AssetID_8343', 'AssetID_8344',
            'AssetID_8346', 'AssetID_9286', 'AssetID_9287',
            'model_value'
        ]

        # حذف ردیف‌هایی که هر یک از این ستون‌ها NaN دارند
        df_features = df[feature_cols].dropna()
        if df_features.empty:
            logging.warning("بعد از حذف NaN هیچ داده‌ای برای SVM باقی نماند.")
            df.to_excel(EXCEL_FILE, index=False)
            print("فقط مرحله اول اجرا شد (داده کافی برای SVM نبود).")
            return

        X = df_features.values

        # استانداردسازی
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # مدل One-Class SVM
        # nu=0.05 یعنی تقریباً ۵٪ داده‌ها را به عنوان outlier در نظر می‌گیرد (قابل تنظیم)
        oc_svm = OneClassSVM(kernel='rbf', gamma='scale', nu=0.05)
        oc_svm.fit(X_scaled)

        # پیش‌بینی: -1 = anomaly, +1 = normal
        anomaly_labels = oc_svm.predict(X_scaled)          # -1 یا +1
        anomaly_scores = oc_svm.decision_function(X_scaled)  # هرچه منفی‌تر → انومالی‌تر

        # اضافه کردن نتایج به دیتافریم اصلی (فقط ردیف‌های بدون NaN)
        df_clean = df.dropna(subset=feature_cols).reset_index(drop=True)
        df_clean['anomaly_score'] = anomaly_scores
        df_clean['svm_anomaly_flag'] = anomaly_labels  # -1 یا +1

        # فقط مواردی که SVM هم آن‌ها را anomaly تشخیص داد (-1)
        final_df = df_clean[df_clean['svm_anomaly_flag'] == -1].copy()

        # مرتب‌سازی بر اساس امتیاز انومالی (از ناهنجارترین به کمتر)
        final_df = final_df.sort_values(by='anomaly_score', ascending=True)

        # حذف ستون‌های کمکی اگر نمی‌خوای تو خروجی نهایی باشن (اختیاری)
        # final_df = final_df.drop(columns=['anomaly_score', 'svm_anomaly_flag'])

        # ذخیره نهایی
        final_df.to_excel(EXCEL_FILE, index=False)

        logging.info(f"{len(final_df)} رکورد پس از دو مرحله فیلتر (مدل اولیه + One-Class SVM) ذخیره شد.")
        print(f"\nموفقیت‌آمیز!")
        print(f"   • مرحله اول (مدل اولیه): {len(df)} رکورد")
        print(f"   • مرحله دوم (One-Class SVM): {len(final_df)} رکورد نهایی")
        print(f"   فایل خروجی: {EXCEL_FILE}")

    except mysql.connector.Error as e:
        logging.error(f"خطا در اتصال به دیتابیس یا اجرای کوئری: {e}")
        print(f"خطا در دیتابیس: {e}")
    except Exception as e:
        logging.error(f"خطای غیرمنتظره: {e}")
        print(f"خطای غیرمنتظره: {e}")
    finally:
        if conn and conn.is_connected():
            conn.close()
            logging.info("اتصال به دیتابیس بسته شد.")

# -------------------------------
# 3. اجرا
# -------------------------------
if __name__ == "__main__":
    start_time = datetime.now()
    logging.info("شروع فرآیند دو مرحله‌ای تشخیص انومالی...")
    print("در حال استخراج و فیلتر دو مرحله‌ای رکوردهای غیرعادی...")

    export_double_filtered_abnormal()

    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()
    logging.info(f"پایان فرآیند. مدت زمان: {duration:.2f} ثانیه")
    print(f"تمام شد. مدت زمان: {duration:.2f} ثانیه")